# Auto Sampler script
## Elveflow Instruments
* OB1 pressure controller
* MUX Wire
* 
## troubleshooting
* when ob1 set pressure is not working, try: 
 * restarting kernel
 * turn off switches on ob1 and unplug replug usb
 
## LOD
* RPMI
 * 10sec integration, 50 spectra per sample

## Import Libraries

%reset -f



## Instrument Class definitions

In [1]:
from sampler_lib import *


import matplotlib.pyplot as plt
import numpy as np

poe = sampler(verbose=True)

poe.set_valves('air', 'flow-cell')
poe.ob1.set_pressure(0)


b'Instrument name and regulator types hardcoded in the python script'
C:\Users\Admin\Documents\GitHub\fluid-auto-sampler\control_scripts\ob1_calibration.txt
0
0


0

In [3]:
poe.clear('tflask3', pressure=150)

Clearing lines to tflask3
pushpull: 150 mbar for 10 secs
pushpull Time          0: current flow rate          1
pushpull Time      0.006: current flow rate       42.4
pushpull Time       1.01: current flow rate       37.4
pushpull Time       2.02: current flow rate       37.2
pushpull Time       3.03: current flow rate       32.2
pushpull Time       4.04: current flow rate         27
pushpull Time       5.05: current flow rate       42.2
pushpull Time       6.06: current flow rate       42.2
pushpull Time       7.07: current flow rate       42.2
pushpull Time       8.07: current flow rate       26.8
pushpull Time       9.08: current flow rate         32
Zeroing pressure
Push pull: done



In [8]:
max_press=70.0
source='tflask3'
poe.set_valves('water', source)
poe.sample(sensor=("msr", 1), setpoint=-100, press_range=(-max_press,max_press/2.0), trigger=0.0, overtime=0.5, timeout=20)

Time      0.007: err=      -101 err_i=     -10.1 y=       0.8 x=     -28.2
Time      0.117: err=     -95.8 err_i=     -19.7 y=      -4.2 x=     -27.8
Time      0.225: err=     -90.4 err_i=     -28.7 y=      -9.6 x=     -28.3
Time      0.334: err=       -85 err_i=     -37.2 y=       -15 x=     -28.6
Time      0.444: err=     -90.4 err_i=     -46.2 y=      -9.6 x=     -31.9
Time      0.555: err=      -106 err_i=     -56.9 y=       6.2 x=     -38.1
Time      0.664: err=     -95.8 err_i=     -66.4 y=      -4.2 x=     -37.1
Time      0.773: err=     -95.6 err_i=       -76 y=      -4.4 x=     -39.1
Time      0.884: err=      -101 err_i=     -86.1 y=       0.8 x=     -42.5
Time      0.994: err=      -101 err_i=     -96.2 y=         1 x=     -44.5
Time       1.11: err=     -95.6 err_i=      -106 y=      -4.4 x=       -45
Time       1.22: err=       -85 err_i=      -114 y=       -15 x=       -44
Time       1.33: err=      -106 err_i=      -125 y=         6 x=     -51.7
Time       1.44: err=    

In [7]:


print('hello!')



In [2]:
%%capture cap --no-stderr

import datetime

samples = ['tflask1', 'tflask2', 'tflask3']
# samples = ['tflask3']
poe.take_measurement(samples=samples, N=50, Nclean=2)

filename = datetime.datetime.now().strftime("Log %b %d %Y %H-%M-%S") + '.txt'
with open(filename, 'w') as f:
    f.write(cap.stdout)


send sample from tflask1 to cell
Sending sample from tflask1 to flowcell
Clearing lines to flow-cell
pushpull: 100 mbar for 5 secs
pushpull Time          0: current flow rate       15.8
pushpull Time      0.008: current flow rate       45.8
pushpull Time       1.02: current flow rate       40.8
pushpull Time       2.03: current flow rate       45.8
pushpull Time       3.03: current flow rate       40.8
pushpull Time       4.04: current flow rate       45.8
Zeroing pressure
Push pull: done

pushpull: -60 mbar for 3 secs
pushpull Time          0: current flow rate       -4.2
pushpull Time      0.009: current flow rate          1
pushpull Time       1.02: current flow rate      -14.8
pushpull Time       2.03: current flow rate       -9.6
Zeroing pressure
Push pull: done

Time       0.01: err=     -90.6 err_i=     -9.06 y=      -9.4 x=     -25.4
Time       0.12: err=     -90.6 err_i=     -18.1 y=      -9.4 x=     -26.3
Time      0.233: err=     -90.6 err_i=     -27.2 y=      -9.4 x=     -

In [3]:
%%capture cap --no-stderr

# cleaning measurements with DI water reservoirs
import datetime

samples = ['tflask1', 'tflask2', 'tflask3']
poe.take_measurement(samples=samples, N=0, Nclean=0)


filename = datetime.datetime.now().strftime("Sampler log %b %d %Y %H-%M-%S") + '.txt'
with open(filename, 'w') as f:
    f.write(cap().stdout)

NameError: name 'cap' is not defined

In [4]:
cap().stdout


send sample from tflask1 to cell
Sending sample from tflask1 to flowcell
Clearing lines to flow-cell
pushpull: 100 mbar for 5 secs
pushpull Time          0: current flow rate         16
pushpull Time      0.005: current flow rate       46.6
pushpull Time       1.01: current flow rate       41.6
pushpull Time       2.02: current flow rate       46.6
pushpull Time       3.03: current flow rate       51.8
pushpull Time       4.04: current flow rate       41.6
Zeroing pressure
Push pull: done

pushpull: -60 mbar for 3 secs
pushpull Time          0: current flow rate       -4.4
pushpull Time      0.008: current flow rate      -15.2
pushpull Time       1.02: current flow rate        -15
pushpull Time       2.02: current flow rate        -15
Zeroing pressure
Push pull: done

Time       0.01: err=      -106 err_i=     -10.6 y=       6.2 x=     -29.7
Time       0.12: err=      -101 err_i=     -20.7 y=       0.8 x=     -29.3
Time       0.23: err=      -106 err_i=     -31.3 y=       6.2 x=     -

AttributeError: 'NoneType' object has no attribute 'stdout'

In [5]:
poe.clean_cell(cycles=1)

Cleaning flow cell
Sending sample from tflask4 to flowcell
Clearing lines to flow-cell
pushpull: 60 mbar for 5 secs
pushpull Time          0: current flow rate          6
pushpull Time      0.008: current flow rate         26
pushpull Time       1.02: current flow rate         46
pushpull Time       2.02: current flow rate         31
pushpull Time       3.03: current flow rate       35.8
pushpull Time       4.04: current flow rate       26.2
Zeroing pressure
Push pull: done

pushpull: -60 mbar for 3 secs
pushpull Time          0: current flow rate        6.2
pushpull Time       0.01: current flow rate       -9.8
pushpull Time       1.02: current flow rate      -20.4
pushpull Time       2.03: current flow rate      -25.6
Zeroing pressure
Push pull: done

Pressure under range -277.25600000000003
Time       0.01: err=      -990 err_i=       -99 y=      -9.8 x=       -70
Pressure under range -290.126
Time      0.121: err=    -1e+03 err_i=      -199 y=       0.8 x=       -70
Pressure under 

In [ ]:
poe.set_valves('water', 'flow-cell')
poe.pushpull(pressure=-50, timeout=10)

In [5]:
poe.clear(target='tflask3', timeout=10, pressure=300)

Clearing lines to tflask3
pushpull: 300 mbar for 10 secs
pushpull Time          0: current flow rate       -9.4
pushpull Time      0.005: current flow rate       52.6
pushpull Time       1.01: current flow rate         32
pushpull Time       2.02: current flow rate         37
pushpull Time       3.03: current flow rate         32
pushpull Time       4.04: current flow rate       26.8
pushpull Time       5.05: current flow rate       21.8
pushpull Time       6.05: current flow rate         32
pushpull Time       7.06: current flow rate         37
pushpull Time       8.07: current flow rate       21.8
pushpull Time       9.08: current flow rate       16.6
Zeroing pressure
Push pull: done



In [3]:
poe.clear(target='flow-cell', timeout=30, pressure=100)

Clearing lines to flow-cell
pushpull: 100 mbar for 30 secs
pushpull Time          0: current flow rate       15.8
pushpull Time      0.007: current flow rate         41
pushpull Time       1.02: current flow rate         36
pushpull Time       2.03: current flow rate         41
pushpull Time       3.03: current flow rate         36
pushpull Time       4.04: current flow rate       40.8
pushpull Time       5.05: current flow rate         46
pushpull Time       6.06: current flow rate       50.8
pushpull Time       7.07: current flow rate       45.8
pushpull Time       8.07: current flow rate       35.8
pushpull Time       9.08: current flow rate       40.8
pushpull Time       10.1: current flow rate       45.8
pushpull Time       11.1: current flow rate       35.8
pushpull Time       12.1: current flow rate       40.8
pushpull Time       13.1: current flow rate       35.8
pushpull Time       14.1: current flow rate       45.8
pushpull Time       15.1: current flow rate       50.8
pushpu

In [3]:
poe.clear(target='flow-cell', timeout=20)

Clearing lines to flow-cell
pushpull: 100 mbar for 20 secs
pushpull Time          0: current flow rate          6
pushpull Time      0.007: current flow rate       20.8
pushpull Time       1.02: current flow rate         26
pushpull Time       2.02: current flow rate       15.8
pushpull Time       3.03: current flow rate         36
pushpull Time       4.04: current flow rate         26
pushpull Time       5.05: current flow rate       15.8
pushpull Time       6.06: current flow rate         21
pushpull Time       7.06: current flow rate    4.6e+03
pushpull Time       8.07: current flow rate   5.26e+03
pushpull Time       9.08: current flow rate    5.5e+03
pushpull Time       10.1: current flow rate   3.11e+03
pushpull Time       11.1: current flow rate        322
pushpull Time       12.1: current flow rate       70.6
pushpull Time       13.1: current flow rate       40.6
pushpull Time       14.1: current flow rate       40.6
pushpull Time       15.1: current flow rate       55.6
pushpu

In [2]:
poe.to_flowcell(source='tflask3')

Sending sample from tflask3 to flowcell
Clearing lines to flow-cell
pushpull: 60 mbar for 5 secs
pushpull Time          0: current flow rate         21
pushpull Time      0.005: current flow rate       36.2
pushpull Time       1.01: current flow rate         26
pushpull Time       2.02: current flow rate         46
pushpull Time       3.03: current flow rate       31.2
pushpull Time       4.04: current flow rate         31
Zeroing pressure
Push pull: done

pushpull: -60 mbar for 3 secs
pushpull Time          0: current flow rate       -9.6
pushpull Time      0.008: current flow rate        -15
pushpull Time       1.02: current flow rate      -20.4
pushpull Time       2.02: current flow rate      -20.4
Zeroing pressure
Push pull: done

Time       0.01: err=     -95.6 err_i=     -9.56 y=      -4.4 x=     -26.8
Time       0.12: err=     -95.6 err_i=     -19.1 y=      -4.4 x=     -27.7
Time      0.232: err=     -90.4 err_i=     -28.2 y=      -9.6 x=     -28.2
Time      0.342: err=      -10

In [9]:
poe.clean_cell(cycles=1)

Cleaning flow cell
Sending sample from tflask4 to flowcell
Clearing lines to flow-cell
pushpull: 60 mbar for 5 secs
pushpull Time          0: current flow rate         11
pushpull Time      0.008: current flow rate         31
pushpull Time       1.02: current flow rate         41
pushpull Time       2.02: current flow rate         31
pushpull Time       3.03: current flow rate         36
pushpull Time       4.04: current flow rate         26
Zeroing pressure
Push pull: done

pushpull: -60 mbar for 3 secs
pushpull Time          0: current flow rate       -4.4
pushpull Time      0.009: current flow rate       -9.8
pushpull Time       1.02: current flow rate       -9.8
pushpull Time       2.03: current flow rate      -20.4
Zeroing pressure
Push pull: done

Pressure under range -278.76800000000003
Time       0.01: err=      -996 err_i=     -99.6 y=      -4.4 x=       -70
Pressure under range -290.18
Time      0.123: err=    -1e+03 err_i=      -200 y=       0.8 x=       -70
Pressure under r

ValueError: Timed out during sampling to ('msr', 2)

In [ ]:
poe.acton.send_trigger(N=5)

In [ ]:
poe.to_flowcell(source='water')

In [ ]:
poe.to_flowcell(source='tflask1')

In [ ]:
poe.to_flowcell(source='water')

In [ ]:
poe.mix(source='tflask4', sink='tflask1', Nplugs=2)

In [ ]:
poe.clean(target='flow-cell')

In [ ]:
poe.clear(target='tflask3', timeout=10)

In [ ]:
poe.set_valves('air', 'flow-cell')

In [ ]:
poe.set_valves('air', 'tflask2')

In [ ]:
poe.ob1.set_pressure(250)

In [ ]:
poe.ob1.set_pressure(0)

In [ ]:
poe.ob1.meas_pressure()

In [ ]:
# Clearing lines
poe.set_valves('air', 'tflask1')
poe.pushpull(pressure=500, timeout=10)
poe.set_valves('air', 'tflask2')
poe.pushpull(pressure=500, timeout=10)


poe.set_valves('water', 'flow-cell')
poe.pushpull(pressure=100, timeout=10, sensor=('msr', 2))

poe.set_valves('air', 'flow-cell')
poe.pushpull(pressure=200, timeout=20, sensor=('msr', 2))

In [ ]:
poe.set_valves('water', 'tflask2')
poe.sample(sensor=("msr", 1), setpoint=-500, trigger=0.1, overtime=10.0, timeout=40)

poe.set_valves('air', 'flow-cell')
poe.sample(sensor=("msr", 2), setpoint=50, press_range=(-70,100), trigger=0.0, overtime=0.0, timeout=50)
# poe.pushpull(pressure=300, timeout=10)

# Clean
# poe.set_valves('water', 'tflask')
# poe.pushpull(pressure=500, timeout=5)
poe.set_valves('air', 'tflask2')
poe.pushpull(pressure=250, timeout=10)

# poe.set_valves('air', 'flow-cell')
# poe.pushpull(pressure=100, timeout=10)

In [ ]:
poe.set_valves('air', 'flow-cell')
poe.pushpull(pressure=50, timeout=30)

In [ ]:
sens = msr()

In [ ]:
del sens

In [ ]:
sens.meas_flowrate(channel=3)

In [ ]:
del sens

In [ ]:
poe.set_valves('air', 'flow-cell')

In [ ]:
poe.set_valves('water', 'flow-cell')

In [ ]:
poe.set_valves('air', 'tflask2')

In [ ]:
poe.set_valves('water', 'tflask')

In [ ]:
poe.set_valves('water', 'flow-cell')

In [ ]:
poe.pushpull(pressure=900, timeout=10)

In [ ]:
poe.ob1.set_pressure(-500)

In [ ]:
poe.ob1.set_pressure(100)

In [ ]:
poe.ob1.set_pressure(0)

In [ ]:
poe.ob1.meas_flowrate()

In [ ]:
poe.msr.meas_flowrate(channel=2)

In [ ]:
poe.ob1.meas_pressure()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

vol = np.array([0.2758,
        0.2753,
        0.2761,
        0.2739,
        0.2761,
        0.2748,
        0.2777,
        0.2782,
        0.2771,
        0.2791])*1e3

print('{} Samples: avg={:6.5g} uL, stdev={:6.5g} uL'.format(vol.shape[0], np.mean(vol), np.std(vol)))

plt.figure()
plt.plot(range(1,11), vol, 'o', label='Data')
plt.plot([1,10], [np.mean(vol)]*2, '-', label='Mean={:.3g} $\pm$ {:.3g} $\mu L$'.format(np.mean(vol), np.std(vol)))
plt.legend()
plt.xlabel('Sample #')
plt.ylabel('$\mu L$')
plt.savefig('')
plt.show()

In [ ]:
from time import sleep, time
import matplotlib.pyplot as plt

poe.set_valves('air', 'tflask')

# PID attempt

Kp = 0.5
Ki = 0.2
Kd = 0.005

setpoint = -200.0 # uL/min

timeout = 10.0 # sec
start_time = time()
cur_time = time()-start_time
err_i = 0.0
err_d = 0.0
err_prev = 0.0
dt = 0.15 # sec

# traces
t_arr = []
x_arr = []
y_arr = []
while cur_time < timeout:
    y,flag = poe.ob1.meas_flowrate()
    if flag == 0:
        err = setpoint - y

        err_i = err_i + err*dt
        err_d = (err - err_prev)/dt

        x = Kp*err + Ki*err_i + Kd*err_d
        
        if x > 1000:
            print('Pressure over range {}'.format(x))
            x = 1000.0
        elif x < -900:
            print('Pressure under range {}'.format(x))
            x = -900.0
            
        poe.ob1.set_pressure(x)
        
        x_arr.append(x)
        y_arr.append(y)
        t_arr.append(cur_time)
        

        err_prev = err
        cur_time = time()-start_time
        print('Time {:5.5g}: err={:5.5g} err_i={:5.5g} y={:5.5g} x={:5.5g}'.format(cur_time, err, err_i, y, x))
    else:
        print('Flow rate sensor error {}, value {}'.format(flag, y))
        
    sleep(dt)

poe.set_valves('air', 'tflask')
poe.ob1.set_pressure(200.0)
sleep(5.0)
poe.ob1.set_pressure(0.0)


# plot results
plt.figure(figsize=(10,3))
plt.subplot(121)
plt.title('Pressure [mbar]')
plt.plot(t_arr, x_arr, label='x')
plt.xlabel('Time [sec]')


plt.subplot(122)
plt.title('Y: flowrate [uL/min]')
plt.plot(t_arr, y_arr, label='y')
plt.plot([t_arr[0],t_arr[-1]], [setpoint, setpoint], label='setpoint')
plt.xlabel('Time [sec]')
plt.legend()
plt.savefig('PID.png', dpi=200)
plt.show()

In [ ]:
print('{:4.3g}'.format(1.1122))

In [ ]:
import matplotlib.pyplot as plt
poe.set_valves('air', 'tflask')
time,vol = poe.pushpull(mode=1, target=30, pressure=-20)
poe.set_valves('air', 'flow-cell')
time_rel = [t-time[0] for t in time]
plt.plot(time_rel, vol)
plt.xlabel('Time [sec]')
plt.ylabel('Volume [uL]')
plt.show()

In [ ]:
del poe

In [ ]:
# kenobi=ob1(calibrate=True)
kenobi=ob1(calibrate=False)

In [ ]:
p,e = kenobi.meas_pressure()

print(p)

In [ ]:
kenobi.set_pressure(-50)

In [ ]:
kenobi.meas_flowrate()

In [ ]:
mux = mux_wire()

In [ ]:
state = [1, 0, 0, 0, 0, 0, 0, 0]
mux.set_all_valves(state)

In [ ]:
isinstance(state, list)

In [ ]:
print(os.path.expanduser(r'./ob1_calibration.txt'))
print(os.getcwd())
print(os.path.relpath('ob1_calibration.txt'))
print(os.path.abspath('ob1_calibration.txt'))

In [ ]:
del kenobi
del mux

In [ ]:
print([0]*8)